## Introduction to deep learning - exercise

## Exercise

*Human Activity Recognition using Smartphones* dataset

Dataset description:

*The experiments have been carried out with a group of 30 volunteers. Each person performed six activities
(WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone.
Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity.
The experiments have been video-recorded to label the data manually.*

**Variables:**
For each record in the dataset it is provided:
* A 561-feature vector with time and frequency domain variables.
* Its activity label.
* An identifier of the subject who carried out the experiment.

More details at: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

### Loading and preparing the data

In [1]:
import pandas as pd
import os

In [2]:
folder = ''  ## put here folder where the file HAR_clean.csv is located

Load the dataset already preprocessed and clean: "HAR_clean.csv"

In [3]:
all_data = pd.read_csv(os.path.join(folder, 'HAR_clean.csv'), index_col=0)

Divide into input and output data

In [4]:
input_data = all_data.iloc[:,:-2]
input_data.shape

(10299, 561)

In [5]:
output_data = all_data.iloc[:,-1]
output_data.shape

(10299,)

Divide the data into train and test, keeping 30% for the test

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.3)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(7209, 561) (7209,)
(3090, 561) (3090,)


### Best shallow ML model

In [7]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':['linear', 'rbf'], 'C':[1, 10, 100,1000], 'gamma':[0.01, 0.001]}

svm_model_d = svm.SVC()
opt_model_d = GridSearchCV(svm_model_d, parameters)

opt_model_d.fit(X_train, y_train)
print (opt_model_d.best_estimator_)

SVC(C=100, gamma=0.01)


In [8]:
opt_model_d.score(X_test, y_test)

0.9906148867313915

### Deep Learning models

**Ex. 1** - Train a Deep Neural Network model for this dataset and compare its performance with the shallow models from previous sessions.

In [9]:
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

le = preprocessing.LabelEncoder()

le.fit(y_train)

y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

2025-02-25 19:06:29.501941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-25 19:06:29.501956: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert labels to categorical (one-hot encoding)
num_classes = len(le.classes_)  # Number of unique labels
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

In [11]:
# Build the deep neural network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2025-02-25 19:06:30.467756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-25 19:06:30.467777: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-25 19:06:30.467795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (PCdoEduardo): /proc/driver/nvidia/version does not exist
2025-02-25 19:06:30.467988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# Train the model
history = model.fit(X_train_scaled, y_train_onehot, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test_onehot))

Epoch 1/50
226/226 [==============================] - 1s 1ms/step - loss: 0.5294 - accuracy: 0.7875 - val_loss: 0.1409 - val_accuracy: 0.9469
Epoch 2/50
226/226 [==============================] - 0s 1ms/step - loss: 0.1913 - accuracy: 0.9294 - val_loss: 0.0888 - val_accuracy: 0.9667
Epoch 3/50
226/226 [==============================] - 0s 1ms/step - loss: 0.1344 - accuracy: 0.9506 - val_loss: 0.0815 - val_accuracy: 0.9686
Epoch 4/50
226/226 [==============================] - 0s 1ms/step - loss: 0.1258 - accuracy: 0.9521 - val_loss: 0.0787 - val_accuracy: 0.9660
Epoch 5/50
226/226 [==============================] - 0s 1ms/step - loss: 0.1041 - accuracy: 0.9624 - val_loss: 0.0852 - val_accuracy: 0.9689
Epoch 6/50
226/226 [==============================] - 0s 1ms/step - loss: 0.0971 - accuracy: 0.9612 - val_loss: 0.1006 - val_accuracy: 0.9560
Epoch 7/50
226/226 [==============================] - 0s 996us/step - loss: 0.0893 - accuracy: 0.9662 - val_loss: 0.0754 - val_accuracy: 0.9689
Epoc

In [13]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_onehot)
print(f"Test Accuracy: {test_acc:.4f}")

97/97 [==============================] - 0s 546us/step - loss: 0.0730 - accuracy: 0.9770
Test Accuracy: 0.9770


**Ex. 2** - Play with the different parameters (topologies, training algorithms, etc) and check their performance.

In [14]:
def create_model(hidden_layers=[128, 64, 32], dropout_rate=0.3, activation='relu', optimizer='adam'):
    model = keras.Sequential()
    model.add(keras.layers.Dense(hidden_layers[0], activation=activation, input_shape=(X_train.shape[1],)))
    
    for units in hidden_layers[1:]:
        model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(units, activation=activation))
    
    model.add(keras.layers.Dense(num_classes, activation='softmax'))  # Output layer

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Experiment with different architectures
topologies = [
    [128, 64, 32],  # Baseline
    [256, 128, 64],  # Deeper model
    [512, 256, 128, 64],  # Very deep model
    [128, 64],  # Simpler model
]

for topology in topologies:
    print(f"\nTesting topology: {topology}")
    model = create_model(hidden_layers=topology)
    model.fit(X_train_scaled, y_train_onehot, epochs=30, batch_size=32, validation_data=(X_test_scaled, y_test_onehot), verbose=1)
    test_loss, test_acc = model.evaluate(X_test_scaled, y_test_onehot)
    print(f"Test Accuracy: {test_acc:.4f}")


Testing topology: [128, 64, 32]
Epoch 1/30
226/226 [==============================] - 1s 1ms/step - loss: 0.5270 - accuracy: 0.7897 - val_loss: 0.1668 - val_accuracy: 0.9307
Epoch 2/30
226/226 [==============================] - 0s 1ms/step - loss: 0.2091 - accuracy: 0.9194 - val_loss: 0.1146 - val_accuracy: 0.9550
Epoch 3/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1484 - accuracy: 0.9449 - val_loss: 0.1025 - val_accuracy: 0.9615
Epoch 4/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1244 - accuracy: 0.9524 - val_loss: 0.1033 - val_accuracy: 0.9621
Epoch 5/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1024 - accuracy: 0.9599 - val_loss: 0.0766 - val_accuracy: 0.9731
Epoch 6/30
226/226 [==============================] - 0s 1ms/step - loss: 0.0938 - accuracy: 0.9635 - val_loss: 0.0723 - val_accuracy: 0.9725
Epoch 7/30
226/226 [==============================] - 0s 1ms/step - loss: 0.0794 - accuracy: 0.9702 - val_loss: 0.0

In [15]:
optimizers = ['adam', 'sgd', 'rmsprop']
for opt in optimizers:
    print(f"\nTesting optimizer: {opt}")
    model = create_model(optimizer=opt)
    model.fit(X_train_scaled, y_train_onehot, epochs=30, batch_size=32, validation_data=(X_test_scaled, y_test_onehot), verbose=1)
    test_loss, test_acc = model.evaluate(X_test_scaled, y_test_onehot)
    print(f"Test Accuracy with {opt}: {test_acc:.4f}")


Testing optimizer: adam
Epoch 1/30
226/226 [==============================] - 1s 1ms/step - loss: 0.5728 - accuracy: 0.7768 - val_loss: 0.1949 - val_accuracy: 0.9207
Epoch 2/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1985 - accuracy: 0.9234 - val_loss: 0.1022 - val_accuracy: 0.9615
Epoch 3/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1603 - accuracy: 0.9385 - val_loss: 0.0976 - val_accuracy: 0.9634
Epoch 4/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1242 - accuracy: 0.9535 - val_loss: 0.1050 - val_accuracy: 0.9576
Epoch 5/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1098 - accuracy: 0.9573 - val_loss: 0.0798 - val_accuracy: 0.9696
Epoch 6/30
226/226 [==============================] - 0s 1ms/step - loss: 0.1022 - accuracy: 0.9625 - val_loss: 0.0829 - val_accuracy: 0.9673
Epoch 7/30
226/226 [==============================] - 0s 1ms/step - loss: 0.0872 - accuracy: 0.9666 - val_loss: 0.0732 - va

In [16]:
batch_sizes = [16, 32, 64]
epochs_list = [20, 50, 100]

for batch in batch_sizes:
    for epochs in epochs_list:
        print(f"\nTesting batch_size={batch}, epochs={epochs}")
        model = create_model()
        model.fit(X_train_scaled, y_train_onehot, epochs=epochs, batch_size=batch, validation_data=(X_test_scaled, y_test_onehot), verbose=1)
        test_loss, test_acc = model.evaluate(X_test_scaled, y_test_onehot)
        print(f"Test Accuracy: {test_acc:.4f}")


Testing batch_size=16, epochs=20
Epoch 1/20
451/451 [==============================] - 1s 1ms/step - loss: 0.4762 - accuracy: 0.8090 - val_loss: 0.1381 - val_accuracy: 0.9515
Epoch 2/20
451/451 [==============================] - 0s 971us/step - loss: 0.2039 - accuracy: 0.9258 - val_loss: 0.1320 - val_accuracy: 0.9450
Epoch 3/20
451/451 [==============================] - 0s 1ms/step - loss: 0.1435 - accuracy: 0.9427 - val_loss: 0.1045 - val_accuracy: 0.9654
Epoch 4/20
451/451 [==============================] - 0s 902us/step - loss: 0.1348 - accuracy: 0.9510 - val_loss: 0.0883 - val_accuracy: 0.9702
Epoch 5/20
451/451 [==============================] - 0s 992us/step - loss: 0.1172 - accuracy: 0.9553 - val_loss: 0.0838 - val_accuracy: 0.9654
Epoch 6/20
451/451 [==============================] - 0s 1ms/step - loss: 0.1095 - accuracy: 0.9591 - val_loss: 0.0902 - val_accuracy: 0.9660
Epoch 7/20
451/451 [==============================] - 1s 1ms/step - loss: 0.0996 - accuracy: 0.9625 - val_lo

**Ex. 3** - Comment on the results obtained !!

tops